In [1]:
import praw
import pandas as pd
import datetime
import numpy as np
from matplotlib import pyplot as plt

/Users/xiaoxiangma/.local/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [152]:
import twint
import pickle
import nltk
# import nest_asyncio
# nest_asyncio.apply()

In [131]:
def get_tweets(search, limit=100):
    c = twint.Config()
    c.Search = search
    c.Limit = limit # If not specified, scrapes all...
    c.Pandas = True
    c.Pandas_clean = True
#     c.Near = "shandong"
#     c.Location = True
    c.Hide_output = True

    c.Store_csv = True
    c.Output = 'test.csv'
    
    twint.run.Search(c)
    return twint.output.panda.Tweets_df

In [172]:
tweets = get_tweets("digital health", limit=3000)

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs


In [173]:
tweets.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username',
       'name', 'day', 'hour', 'link', 'retweet', 'nlikes', 'nreplies',
       'nretweets', 'quote_url', 'search', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [174]:
# pd.set_option("max_columns", None)

# pd.reset_option("max_rows")


In [175]:
tweets

,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1268266714409598978,1268266714409598978,1591213322000,2020-06-03 15:42:02,EDT,,🔹Purposefully design your clinical workflows w...,"[#digitalhealth, #telehealth, #systemsengineer...",[],3242616505,3242616505,Ingenium_HA,Ingenium Healthcare Advisors,3,15,https://twitter.com/Ingenium_HA/status/1268266...,False,0,0,0,,digital health,,,,,,,"[{'user_id': '3242616505', 'username': 'Ingeni...",,,,
1,1268266402575761414,1268266402575761414,1591213248000,2020-06-03 15:40:48,EDT,,"Join our webinar on June 24th, where we will d...","[#patientintake, #informedconsent, #telehealth...",[],161713449,161713449,InterlaceHealth,Interlace Health,3,15,https://twitter.com/InterlaceHealth/status/126...,False,0,0,0,,digital health,,,,,,,"[{'user_id': '161713449', 'username': 'Interla...",,,,
2,1268265449583783936,1268265449583783936,1591213021000,2020-06-03 15:37:01,EDT,,"“Leveraging digital strategy, closely paired w...","[#healthcare, #digitalhealth]",[],1115140063,1115140063,DonnaKLencki,Donna K. Lencki,3,15,https://twitter.com/DonnaKLencki/status/126826...,False,0,0,0,,digital health,,,,,,,"[{'user_id': '1115140063', 'username': 'DonnaK...",,,,
3,1268265224148221952,1268265224148221952,1591212967000,2020-06-03 15:36:07,EDT,,CipherHealth Named “Best Overall Patient Engag...,"[#cure, #digitalhealth, #healthcare, #infograp...",[],226310002,226310002,IAM__Network,IAM Platform,3,15,https://twitter.com/IAM__Network/status/126826...,False,0,0,0,,digital health,,,,,,,"[{'user_id': '226310002', 'username': 'IAM__Ne...",,,,
4,1268265074973716480,1268265074973716480,1591212931000,2020-06-03 15:35:31,EDT,,A lot of people have been asking us about digi...,"[#healthcare, #digitalhealth]",[],355428046,355428046,hax_co,HAX,3,15,https://twitter.com/hax_co/status/126826507497...,False,0,0,0,,digital health,,,,,,,"[{'user_id': '355428046', 'username': 'hax_co'}]",,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1266376891629334529,1266376891629334529,1590762753000,2020-05-29 10:32:33,EDT,,"Virtual care may the new standard, but reimbur...","[#covid19, #maternitycare, #virtualcare, #digi...",[],2904233621,2904233621,Babyscripts,Babyscripts,5,10,https://twitter.com/Babyscripts/status/1266376...,False,3,0,1,,digital health,,,,,,,"[{'user_id': '2904233621', 'username': 'Babysc...",,,,
2996,1266376671252238337,1266376671252238337,1590762701000,2020-05-29 10:31:41,EDT,,Enlisting Technology to Address the Looming #B...,"[#behavioralhealth, #digitalhealth]",[],632988723,632988723,BrandonAylward,Brandon Aylward,5,10,https://twitter.com/BrandonAylward/status/1266...,False,0,0,0,,digital health,,,,,,,"[{'user_id': '632988723', 'username': 'Brandon...",,,,
2997,1266376633553842177,1266376633553842177,1590762692000,2020-05-29 10:31:32,EDT,,"Hybrid brick-and-mortar, virtual care women's ...","[#digitalhealth, #mhealth, #healthtech]",[],48971936,48971936,GaryMonk,Gary Monk,5,10,https://twitter.com/GaryMonk/status/1266376633...,False,0,0,2,,digital health,,,,,,,"[{'user_id': '48971936', 'username': 'GaryMonk'}]",,,,
2998,1266376389613064193,1266376389613064193,1590762634000,2020-05-29 10:30:34,EDT,,If you’re outside #London you can still benefi...,"[#london, #england, #prescription, #pharmacy, ...",[],902496357272965120,902496357272965120,WeArePhlo,Phlo,5,10,https://twitter.com/WeArePhlo/status/126637638...,False,4,0,1,,digital health,,,,,,,"[{'user_id': '902496357272965120', 'username':...",,,,


In [176]:
tweets['tweet']

0       🔹Purposefully design your clinical workflows w...
1       Join our webinar on June 24th, where we will d...
2       “Leveraging digital strategy, closely paired w...
3       CipherHealth Named “Best Overall Patient Engag...
4       A lot of people have been asking us about digi...
                              ...                        
2995    Virtual care may the new standard, but reimbur...
2996    Enlisting Technology to Address the Looming #B...
2997    Hybrid brick-and-mortar, virtual care women's ...
2998    If you’re outside #London you can still benefi...
2999    We have some exciting news to share with you s...
Name: tweet, Length: 3000, dtype: object

In [165]:
import json
with open('gender/top_words.json') as json_file:
    top_words = json.load(json_file)

In [166]:
def find_features(top_words, text):
    feature = {}
    for word in top_words:
        feature[word] = word in text.lower()
    return feature

In [167]:
NB_classifier = pickle.load(open('gender/gender_NB_classifier.sav', 'rb'))

In [168]:
tweets['tweet'][0]

'Five things #COVID is teaching us about the #patient experience:  http://ow.ly/oG6L50zXV91\xa0 via @PhysiciansPract\n#HCIT #digitalhealth #mhealth pic.twitter.com/ZRLydP4dN6'

In [169]:
NB_classifier.classify(find_features(top_words, tweets['tweet'][0]))

'brand'

In [177]:
tweets['gender'] = tweets['tweet'].apply(lambda x: NB_classifier.classify(find_features(top_words, x)))

In [179]:
tweets['gender'].value_counts()

brand     2754
male       220
female      26
Name: gender, dtype: int64